In [ ]:
# pip install meteostat

In [ ]:
from keras.src import optimizers
from keras.src.utils import plot_model
from keras.src.models import Sequential, Model
from keras.src.layers.convolutional.conv1d import Conv1D
from keras.src.layers.pooling.max_pooling1d import MaxPooling1D
from keras.src.layers import Dense, LSTM, RepeatVector, TimeDistributed, Flatten
from keras.src import metrics
from keras.layers import Input
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, date
from sklearn.model_selection import train_test_split

import math
from sklearn.preprocessing import MinMaxScaler

from datetime import datetime as dt, date, timedelta
# from meteostat import Point, Daily

In [ ]:
data = pd.read_csv('/content/preprocessed_main_data_dki1_kemayoran.csv')
data = data.set_index('tanggal')
data

,index,pm10,pm25,so2,co,o3,no2,Tavg_km,RH_avg_km,RR_km,ss_km,ff_avg_km,DDD_CAR_km
tanggal,,,,,,,,,,,,,
2023-01-01,0.0,44.0,55.0,47.0,10.0,24.0,9.0,26.3,87.0,14.5,0.0,1.0,1.0
2023-01-02,1.0,32.0,43.0,52.0,9.0,24.0,8.0,27.5,81.0,31.5,0.7,2.0,9.0
2023-01-03,2.0,31.0,35.0,49.0,9.0,12.0,7.0,26.6,82.0,0.5,0.0,1.0,1.0
2023-01-04,3.0,30.0,47.0,53.0,11.0,15.0,9.0,26.4,86.0,2.4,2.1,1.0,1.0
2023-01-05,4.0,38.0,50.0,50.0,13.0,26.0,11.0,27.4,83.0,35.3,4.9,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-27,726.0,53.0,83.0,12.0,10.0,11.0,36.0,29.2,78.0,7.5,1.2,2.0,1.0
2024-12-28,727.0,64.0,101.0,14.0,9.0,11.0,35.0,30.0,70.0,0.6,3.8,2.0,1.0
2024-12-29,728.0,51.0,21.0,13.0,10.0,13.0,35.0,29.7,73.0,0.0,3.0,1.0,1.0


In [ ]:
WINDOW_SIZE = 7
HORIZON = 7

scaler = None
scaled_data = None
training_data_len = None

In [ ]:
def make_window(data):
    global scaler
    global scaled_data
    global training_data_len

    features = [
        'pm10', 'so2', 'co', 'o3', 'no2', 'Tavg_km',
      'RH_avg_km', 'RR_km', 'ss_km', 'ff_avg_km', 'DDD_CAR_km']

    targets = ['pm25']


    scaler = MinMaxScaler(feature_range=(0,1))
    data[features] = scaler.fit_transform(data[features])
    data[targets] = scaler.fit_transform(data[targets])

    data_X = data[features].values
    data_Y = data[targets].values


    X = []
    y = []

    for i in range(len(data) - WINDOW_SIZE - HORIZON + 1):
        X.append(data_X[i:i+WINDOW_SIZE])
        y.append(data_Y[i+WINDOW_SIZE:i+WINDOW_SIZE + HORIZON])

    training_data_len = math.ceil(len(X)* 0.8)

    #splitting train
    x_train = X[:training_data_len]
    y_train = y[:training_data_len]

    # Val set
    val_test_len = len(X[training_data_len:])
    val_len = math.ceil(val_test_len/2)

    # x_val_test_set = X[training_data_len: training_data_len+val_len]
    # y_val_test_set = y[training_data_len: training_data_len+val_len]
    # # val_len = int(len(x_val_test_set)/2)

    # # Val set
    # x_val = x_val_test_set[: val_len]
    # y_val = y_val_test_set[: val_len]
    x_val = X[training_data_len: training_data_len+val_len]
    y_val = y[training_data_len: training_data_len+val_len]

    # # Test set
    # x_test = x_val_test_set[val_len:]
    # y_test = y_val_test_set[val_len:]

    # Test set
    x_test = X[training_data_len+val_len: ]
    y_test = y[training_data_len+val_len: ]

    # return training_data_len, val_test_len, val_len, x_val_test_set, y_val_test_set

    # Reshaping
    x_train, y_train = np.array(x_train), np.array(y_train)
    # x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

    x_val, y_val = np.array(x_val), np.array(y_val)
    # x_val = np.reshape(x_val, (x_val.shape[0], x_val.shape[1], 1))

    x_test, y_test = np.array(x_test), np.array(y_test)
    # x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

    return x_train, y_train, x_val, y_val, x_test, y_test

In [ ]:
x_train, y_train, x_val, y_val, x_test, y_test = make_window(data)

In [ ]:
x_train.shape, x_val.shape, x_test.shape

((575, 7, 11), (72, 7, 11), (71, 7, 11))

In [ ]:
y_train.shape, y_val.shape, y_test.shape

((575, 7, 1), (72, 7, 1), (71, 7, 1))

# Define Evaluate Error

In [ ]:
from math import sqrt
import tensorflow as tf
from sklearn.metrics import mean_squared_error, mean_absolute_error

# MASE implemented courtesy of sktime - https://github.com/alan-turing-institute/sktime/blob/ee7a06843a44f4aaec7582d847e36073a9ab0566/sktime/performance_metrics/forecasting/_functions.py#L16
def mean_absolute_scaled_error(y_true, y_pred):
  """
  Implement MASE (assuming no seasonality of data).
  """
  mae = tf.reduce_mean(tf.abs(y_true - y_pred))

  # Find MAE of naive forecast (no seasonality)
  mae_naive_no_season = tf.reduce_mean(tf.abs(y_true[1:] - y_true[:-1])) # our seasonality is 1 day (hence the shifting of 1 day)

  return mae / mae_naive_no_season

def mean_absolute_percentage_error(y_true, y_pred):
  y_true, y_pred = np.array(y_true), np.array(y_pred)
  return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def evaluate_preds(y_true, y_pred):
  # Make sure float32 (for metric calculations)
  # y_true = tf.cast(y_true, dtype=tf.float32)
  # y_pred = tf.cast(y_pred, dtype=tf.float32)

  # Calculate various metrics
  mape = mean_absolute_percentage_error(y_true, y_pred)

  y_pred = y_pred.reshape(y_pred.shape[0], -1)
  y_true = y_true.reshape(y_true.shape[0], -1)

  mae = mean_absolute_error(y_true, y_pred)
  mse = mean_squared_error(y_true, y_pred) # puts and emphasis on outliers (all errors get squared)
  # rmse = sqrt(mse)
  # mase = mean_absolute_scaled_error(y_true, y_pred)
  # print(mae, mse, rmse, mape, mase)

  return {
           "mae": mae,
          "mse": mse,
          # "rmse": rmse,
          "mape": mape
          # "mase": mase.numpy()
          }

# CNN Only

## model 1 (64)

In [ ]:
lr = 0.0003

model_cnn_1 = Sequential()
model_cnn_1.add(Conv1D(filters=64, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_cnn_1.add(Dense(1))
model_cnn_1.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_cnn_1.summary()

Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_11 (Conv1D)              │ (None, 7, 64)          │           768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 7, 1)           │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 833 (3.25 KB)

 Trainable params: 833 (3.25 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_cnn_1.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.9600 - mean_absolute_error: 0.9459 - mean_absolute_percentage_error: 171.8285 - root_mean_squared_error: 0.9767 - val_loss: 0.7131 - val_mean_absolute_error: 0.8393 - val_mean_absolute_percentage_error: 128.0792 - val_root_mean_squared_error: 0.8445
Epoch 2/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.3427 - mean_absolute_error: 0.5378 - mean_absolute_percentage_error: 93.4658 - root_mean_squared_error: 0.5833 - val_loss: 0.2224 - val_mean_absolute_error: 0.4606 - val_mean_absolute_percentage_error: 69.3427 - val_root_mean_squared_error: 0.4716
Epoch 3/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0923 - mean_absolute_error: 0.2597 - mean_absolute_percentage_error: 50.2662 - root_mean_squared_error: 0.3028 - val_loss: 0.0437 - val_mean_absolute_error: 0.1820 - val_mean_absolute_percentage_error: 26.3597 - val_root_mean_squared_error: 0.2091
Epoch 4/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0500 - mean_a

In [ ]:
y_pred = model_cnn_1.predict(x_test)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step


In [ ]:
loss, mae, mape, rmse = model_cnn_1.evaluate(x_test, y_test)
print("model cnn 1")
print("Mean Absolute Error:", mae)
print("Mean Square Error:", loss)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0304 - mean_absolute_error: 0.1344 - mean_absolute_percentage_error: 950024.3125 - root_mean_squared_error: 0.1737
model cnn 1
Mean Absolute Error: 0.1446235328912735
Mean Square Error: 0.034323032945394516
Mean Absolute Percentage Error: 1900020.75
Root Mean Squared Error: 0.18526476621627808


In [ ]:
evaluate_preds(y_test, y_pred)

<ipython-input-10-af04d3513049>:19: RuntimeWarning: divide by zero encountered in divide
  return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


{'mae': 0.14462352648792884,
 'mse': 0.034323030846736234,
 'mape': np.float64(inf)}

## model 2 (128 & 32)
 erornya tinggi

In [ ]:
lr = 0.0003

model_cnn_2 = Sequential()
model_cnn_2.add(Conv1D(filters=128, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_cnn_2.add(Conv1D(filters=32, kernel_size=1, activation='relu'))
model_cnn_2.add(Dense(1))
model_cnn_2.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_cnn_2.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_12 (Conv1D)              │ (None, 7, 128)         │         1,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_13 (Conv1D)              │ (None, 7, 32)          │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 7, 1)           │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,697 (22.25 KB)

 Trainable params: 5,697 (22.25 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_cnn_2.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - loss: 0.5918 - mean_absolute_error: 0.7165 - mean_absolute_percentage_error: 133.1606 - root_mean_squared_error: 0.7613 - val_loss: 0.1056 - val_mean_absolute_error: 0.3120 - val_mean_absolute_percentage_error: 46.4931 - val_root_mean_squared_error: 0.3249
Epoch 2/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0647 - mean_absolute_error: 0.2120 - mean_absolute_percentage_error: 41.1056 - root_mean_squared_error: 0.2531 - val_loss: 0.0101 - val_mean_absolute_error: 0.0838 - val_mean_absolute_percentage_error: 13.0982 - val_root_mean_squared_error: 0.1006
Epoch 3/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0386 - mean_absolute_error: 0.1569 - mean_absolute_percentage_error: 44.2143 - root_mean_squared_error: 0.1964 - val_loss: 0.0101 - val_mean_absolute_error: 0.0844 - val_mean_absolute_percentage_error: 13.2888 - val_root_mean_squared_error: 0.1005
Epoch 4/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0301 - mean_ab

In [ ]:
y_pred = model_cnn_2.predict(x_test)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step


In [ ]:
loss, mae, mape, rmse = model_cnn_2.evaluate(x_test, y_test)
print("model cnn 2")
print("Mean Absolute Error:", mae)
print("Mean Square Error:", loss)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.0306 - mean_absolute_error: 0.1360 - mean_absolute_percentage_error: 891473.1250 - root_mean_squared_error: 0.1743
model cnn 2
Mean Absolute Error: 0.1474056839942932
Mean Square Error: 0.03505583107471466
Mean Absolute Percentage Error: 1782919.125
Root Mean Squared Error: 0.18723201751708984


## model 3 (64 64)

In [ ]:
lr = 0.0003

model_cnn_3 = Sequential()
model_cnn_3.add(Conv1D(filters=64, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_cnn_3.add(Conv1D(filters=64, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_cnn_3.add(Dense(1))
model_cnn_3.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_cnn_3.summary()

Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_14 (Conv1D)              │ (None, 7, 64)          │           768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_15 (Conv1D)              │ (None, 7, 64)          │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 7, 1)           │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,993 (19.50 KB)

 Trainable params: 4,993 (19.50 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_cnn_3.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - loss: 0.2019 - mean_absolute_error: 0.3969 - mean_absolute_percentage_error: 72.5371 - root_mean_squared_error: 0.4433 - val_loss: 0.0179 - val_mean_absolute_error: 0.1108 - val_mean_absolute_percentage_error: 18.2962 - val_root_mean_squared_error: 0.1338
Epoch 2/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0435 - mean_absolute_error: 0.1634 - mean_absolute_percentage_error: 45.9667 - root_mean_squared_error: 0.2084 - val_loss: 0.0113 - val_mean_absolute_error: 0.0924 - val_mean_absolute_percentage_error: 14.6525 - val_root_mean_squared_error: 0.1063
Epoch 3/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0263 - mean_absolute_error: 0.1320 - mean_absolute_percentage_error: 32.5473 - root_mean_squared_error: 0.1621 - val_loss: 0.0133 - val_mean_absolute_error: 0.0962 - val_mean_absolute_percentage_error: 14.4012 - val_root_mean_squared_error: 0.1153
Epoch 4/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0234 - mean_ab

In [ ]:
y_pred = model_cnn_3.predict(x_test)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step


In [ ]:
loss, mae, mape, rmse = model_cnn_3.evaluate(x_test, y_test)
print("model cnn 3")
print("Mean Absolute Error:", mae)
print("Mean Square Error:", loss)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0293 - mean_absolute_error: 0.1308 - mean_absolute_percentage_error: 981607.8125 - root_mean_squared_error: 0.1702
model cnn 3
Mean Absolute Error: 0.1436699479818344
Mean Square Error: 0.034257251769304276
Mean Absolute Percentage Error: 1963190.25
Root Mean Squared Error: 0.1850871443748474


## model 4 (256 64)

In [ ]:
lr = 0.0003

model_cnn_4 = Sequential()
model_cnn_4.add(Conv1D(filters=256, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_cnn_4.add(Conv1D(filters=64, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_cnn_4.add(Dense(1))
model_cnn_4.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_cnn_4.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_27"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_23 (Conv1D)              │ (None, 7, 256)         │         3,072 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_24 (Conv1D)              │ (None, 7, 64)          │        16,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 7, 1)           │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,585 (76.50 KB)

 Trainable params: 19,585 (76.50 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_cnn_4.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - loss: 0.2546 - mean_absolute_error: 0.4366 - mean_absolute_percentage_error: 79.7108 - root_mean_squared_error: 0.4940 - val_loss: 0.0265 - val_mean_absolute_error: 0.1348 - val_mean_absolute_percentage_error: 22.5587 - val_root_mean_squared_error: 0.1629
Epoch 2/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0415 - mean_absolute_error: 0.1596 - mean_absolute_percentage_error: 45.2408 - root_mean_squared_error: 0.2030 - val_loss: 0.0179 - val_mean_absolute_error: 0.1105 - val_mean_absolute_percentage_error: 15.9138 - val_root_mean_squared_error: 0.1339
Epoch 3/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0285 - mean_absolute_error: 0.1390 - mean_absolute_percentage_error: 31.6425 - root_mean_squared_error: 0.1686 - val_loss: 0.0108 - val_mean_absolute_error: 0.0882 - val_mean_absolute_percentage_error: 13.9272 - val_root_mean_squared_error: 0.1037
Epoch 4/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0231 - mean_a

In [ ]:
y_pred = model_cnn_4.predict(x_test)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step


In [ ]:
loss, mae, mape, rmse = model_cnn_4.evaluate(x_test, y_test)
print("model cnn 4")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0332 - mean_absolute_error: 0.1393 - mean_absolute_percentage_error: 730995.4375 - root_mean_squared_error: 0.1816
model cnn 4
Mean Square Error: 0.03757407143712044
Mean Absolute Error: 0.15015512704849243
Mean Absolute Percentage Error: 1461963.5
Root Mean Squared Error: 0.19384032487869263


# LSTM Only

## model 1 (LSTM=32)

In [ ]:
lr = 0.0003

model_lstm_1 = Sequential()
model_lstm_1.add(Input(shape=(x_train.shape[1], x_train.shape[2])))
model_lstm_1.add(LSTM(32, activation='relu'))
model_lstm_1.add(Dense(1))
model_lstm_1.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_lstm_1.summary()

Model: "sequential_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_9 (LSTM)                   │ (None, 32)             │         5,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,665 (22.13 KB)

 Trainable params: 5,665 (22.13 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_lstm_1.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 3s 44ms/step - loss: 0.3830 - mean_absolute_error: 0.5898 - mean_absolute_percentage_error: 106.6195 - root_mean_squared_error: 0.6177 - val_loss: 0.1662 - val_mean_absolute_error: 0.3974 - val_mean_absolute_percentage_error: 59.6184 - val_root_mean_squared_error: 0.4077
Epoch 2/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.1046 - mean_absolute_error: 0.2799 - mean_absolute_percentage_error: 49.2265 - root_mean_squared_error: 0.3212 - val_loss: 0.0163 - val_mean_absolute_error: 0.1055 - val_mean_absolute_percentage_error: 17.6175 - val_root_mean_squared_error: 0.1278
Epoch 3/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0307 - mean_absolute_error: 0.1389 - mean_absolute_percentage_error: 39.0163 - root_mean_squared_error: 0.1750 - val_loss: 0.0201 - val_mean_absolute_error: 0.1159 - val_mean_absolute_percentage_error: 19.4710 - val_root_mean_squared_error: 0.1419
Epoch 4/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0221 - mean

In [ ]:
y_pred = model_lstm_1.predict(x_test)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step


In [ ]:
loss, mae, mape, rmse = model_lstm_1.evaluate(x_test, y_test)
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0283 - mean_absolute_error: 0.1322 - mean_absolute_percentage_error: 768039.4375 - root_mean_squared_error: 0.1677
Mean Square Error: 0.031493570655584335
Mean Absolute Error: 0.14127062261104584
Mean Absolute Percentage Error: 1536051.875
Root Mean Squared Error: 0.17746427655220032


## model 2 (LSTM=32. 32)

In [ ]:
lr = 0.0003

model_lstm_2 = Sequential()
model_lstm_2.add(Input(shape=(x_train.shape[1], x_train.shape[2])))
model_lstm_2.add(LSTM(32, activation='relu', return_sequences=True))
model_lstm_2.add(LSTM(32, activation='relu'))
model_lstm_2.add(Dense(1))
model_lstm_2.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_lstm_2.summary()

Model: "sequential_17"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_12 (LSTM)                  │ (None, 7, 32)          │         5,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_13 (LSTM)                  │ (None, 32)             │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,985 (54.63 KB)

 Trainable params: 13,985 (54.63 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_lstm_2.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 57ms/step - loss: 0.2365 - mean_absolute_error: 0.4518 - mean_absolute_percentage_error: 78.8555 - root_mean_squared_error: 0.4852 - val_loss: 0.1009 - val_mean_absolute_error: 0.3046 - val_mean_absolute_percentage_error: 45.2663 - val_root_mean_squared_error: 0.3177
Epoch 2/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0464 - mean_absolute_error: 0.1756 - mean_absolute_percentage_error: 39.6472 - root_mean_squared_error: 0.2145 - val_loss: 0.0103 - val_mean_absolute_error: 0.0835 - val_mean_absolute_percentage_error: 12.4818 - val_root_mean_squared_error: 0.1016
Epoch 3/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0265 - mean_absolute_error: 0.1328 - mean_absolute_percentage_error: 33.0299 - root_mean_squared_error: 0.1626 - val_loss: 0.0222 - val_mean_absolute_error: 0.1243 - val_mean_absolute_percentage_error: 17.7441 - val_root_mean_squared_error: 0.1490
Epoch 4/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0245 - mean_

In [ ]:
y_pred = model_lstm_2.predict(x_test)

3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 256ms/step


In [ ]:
loss, mae, mape, rmse = model_lstm_2.evaluate(x_test, y_test)
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0293 - mean_absolute_error: 0.1370 - mean_absolute_percentage_error: 721152.4375 - root_mean_squared_error: 0.1704
Mean Square Error: 0.03399365022778511
Mean Absolute Error: 0.14878639578819275
Mean Absolute Percentage Error: 1442276.875
Root Mean Squared Error: 0.184373676776886


## model 3 (LSTM=256. 32)

In [ ]:
lr = 0.0003

model_lstm_3 = Sequential()
model_lstm_3.add(Input(shape=(x_train.shape[1], x_train.shape[2])))
model_lstm_3.add(LSTM(256, activation='relu', return_sequences=True))
model_lstm_3.add(LSTM(32, activation='relu'))
model_lstm_3.add(Dense(1))
model_lstm_3.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_lstm_3.summary()

Model: "sequential_18"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_14 (LSTM)                  │ (None, 7, 256)         │       274,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_15 (LSTM)                  │ (None, 32)             │        36,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 311,457 (1.19 MB)

 Trainable params: 311,457 (1.19 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_lstm_3.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 7s 121ms/step - loss: 0.2062 - mean_absolute_error: 0.4060 - mean_absolute_percentage_error: 73.8303 - root_mean_squared_error: 0.4489 - val_loss: 0.0197 - val_mean_absolute_error: 0.1153 - val_mean_absolute_percentage_error: 19.3943 - val_root_mean_squared_error: 0.1402
Epoch 2/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - loss: 0.0258 - mean_absolute_error: 0.1302 - mean_absolute_percentage_error: 32.0922 - root_mean_squared_error: 0.1605 - val_loss: 0.0091 - val_mean_absolute_error: 0.0788 - val_mean_absolute_percentage_error: 12.6523 - val_root_mean_squared_error: 0.0955
Epoch 3/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - loss: 0.0210 - mean_absolute_error: 0.1130 - mean_absolute_percentage_error: 30.9737 - root_mean_squared_error: 0.1448 - val_loss: 0.0095 - val_mean_absolute_error: 0.0814 - val_mean_absolute_percentage_error: 12.3410 - val_root_mean_squared_error: 0.0973
Epoch 4/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - loss: 0.0200 - mean

In [ ]:
y_pred = model_lstm_3.predict(x_test)

3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 167ms/step


In [ ]:
loss, mae, mape, rmse = model_lstm_3.evaluate(x_test, y_test)
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0270 - mean_absolute_error: 0.1274 - mean_absolute_percentage_error: 678019.0000 - root_mean_squared_error: 0.1636
Mean Square Error: 0.030799882486462593
Mean Absolute Error: 0.13815176486968994
Mean Absolute Percentage Error: 1356012.625
Root Mean Squared Error: 0.17549894750118256


## model 4 (LSTM=256. 64)

In [ ]:
lr = 0.0003

model_lstm_4 = Sequential()
model_lstm_4.add(Input(shape=(x_train.shape[1], x_train.shape[2])))
model_lstm_4.add(LSTM(256, activation='relu', return_sequences=True))
model_lstm_4.add(LSTM(64, activation='relu'))
model_lstm_4.add(Dense(1))
model_lstm_4.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_lstm_4.summary()

Model: "sequential_19"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_16 (LSTM)                  │ (None, 7, 256)         │       274,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_17 (LSTM)                  │ (None, 64)             │        82,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 356,673 (1.36 MB)

 Trainable params: 356,673 (1.36 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_lstm_4.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 5s 81ms/step - loss: 0.1734 - mean_absolute_error: 0.3521 - mean_absolute_percentage_error: 66.0698 - root_mean_squared_error: 0.4076 - val_loss: 0.0183 - val_mean_absolute_error: 0.1119 - val_mean_absolute_percentage_error: 15.9896 - val_root_mean_squared_error: 0.1354
Epoch 2/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - loss: 0.0317 - mean_absolute_error: 0.1481 - mean_absolute_percentage_error: 31.1813 - root_mean_squared_error: 0.1779 - val_loss: 0.0136 - val_mean_absolute_error: 0.0937 - val_mean_absolute_percentage_error: 15.7282 - val_root_mean_squared_error: 0.1166
Epoch 3/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - loss: 0.0226 - mean_absolute_error: 0.1174 - mean_absolute_percentage_error: 32.3151 - root_mean_squared_error: 0.1504 - val_loss: 0.0096 - val_mean_absolute_error: 0.0823 - val_mean_absolute_percentage_error: 12.4434 - val_root_mean_squared_error: 0.0982
Epoch 4/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - loss: 0.0193 - mean_

In [ ]:
y_pred = model_lstm_4.predict(x_test)

3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 168ms/step


In [ ]:
loss, mae, mape, rmse = model_lstm_4.evaluate(x_test, y_test)
print("LSTM Only 256 & 64")
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.0302 - mean_absolute_error: 0.1248 - mean_absolute_percentage_error: 534751.6250 - root_mean_squared_error: 0.1732
Mean Square Error: 0.0356367826461792
Mean Absolute Error: 0.1373991221189499
Mean Absolute Percentage Error: 1069481.25
Root Mean Squared Error: 0.18877707421779633


## model 5 (LSTM=512. 128)

In [ ]:
lr = 0.0003

model_lstm_5 = Sequential()
model_lstm_5.add(Input(shape=(x_train.shape[1], x_train.shape[2])))
model_lstm_5.add(LSTM(512, activation='relu', return_sequences=True))
model_lstm_5.add(LSTM(128, activation='relu'))
model_lstm_5.add(Dense(1))
model_lstm_5.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_lstm_5.summary()

Model: "sequential_20"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_18 (LSTM)                  │ (None, 7, 512)         │     1,073,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_19 (LSTM)                  │ (None, 128)            │       328,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,401,473 (5.35 MB)

 Trainable params: 1,401,473 (5.35 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_lstm_5.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 7s 238ms/step - loss: 0.1826 - mean_absolute_error: 0.3640 - mean_absolute_percentage_error: 67.8239 - root_mean_squared_error: 0.4175 - val_loss: 0.0331 - val_mean_absolute_error: 0.1594 - val_mean_absolute_percentage_error: 22.9292 - val_root_mean_squared_error: 0.1818
Epoch 2/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 130ms/step - loss: 0.0345 - mean_absolute_error: 0.1530 - mean_absolute_percentage_error: 33.0749 - root_mean_squared_error: 0.1852 - val_loss: 0.0092 - val_mean_absolute_error: 0.0803 - val_mean_absolute_percentage_error: 12.1984 - val_root_mean_squared_error: 0.0961
Epoch 3/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 119ms/step - loss: 0.0218 - mean_absolute_error: 0.1193 - mean_absolute_percentage_error: 29.4415 - root_mean_squared_error: 0.1475 - val_loss: 0.0094 - val_mean_absolute_error: 0.0795 - val_mean_absolute_percentage_error: 12.9086 - val_root_mean_squared_error: 0.0971
Epoch 4/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 120ms/step - loss: 0.0191 - m

In [ ]:
y_pred = model_lstm_5.predict(x_test)

3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 493ms/step


In [ ]:
loss, mae, mape, rmse = model_lstm_5.evaluate(x_test, y_test)
print("Mean Square Error:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.0304 - mean_absolute_error: 0.1366 - mean_absolute_percentage_error: 624910.3750 - root_mean_squared_error: 0.1735
Mean Square Error: 0.03534186631441116
Mean Absolute Error: 0.149057999253273
Mean Absolute Percentage Error: 1249793.625
Root Mean Squared Error: 0.1879943311214447


# Hybrid

## model 1 (CNN=64 LSTM=64)

In [ ]:
lr = 0.0003

model_hybrid_1 = Sequential()
model_hybrid_1.add(Conv1D(filters=64, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
# model_hybrid_1.add(MaxPooling1D(pool_size=1))
model_hybrid_1.add(LSTM(64, activation='tanh', return_sequences=True))
model_hybrid_1.add(Dense(1))
model_hybrid_1.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_hybrid_1.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_21"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_17 (Conv1D)              │ (None, 7, 64)          │           768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_20 (LSTM)                  │ (None, 7, 64)          │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 7, 1)           │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 33,857 (132.25 KB)

 Trainable params: 33,857 (132.25 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_hybrid_1.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 44ms/step - loss: 0.2790 - mean_absolute_error: 0.4757 - mean_absolute_percentage_error: 87.0252 - root_mean_squared_error: 0.5224 - val_loss: 0.0721 - val_mean_absolute_error: 0.2274 - val_mean_absolute_percentage_error: 35.4376 - val_root_mean_squared_error: 0.2685
Epoch 2/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0817 - mean_absolute_error: 0.2314 - mean_absolute_percentage_error: 55.4840 - root_mean_squared_error: 0.2854 - val_loss: 0.0554 - val_mean_absolute_error: 0.1853 - val_mean_absolute_percentage_error: 27.3955 - val_root_mean_squared_error: 0.2354
Epoch 3/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0544 - mean_absolute_error: 0.1870 - mean_absolute_percentage_error: 39.7639 - root_mean_squared_error: 0.2332 - val_loss: 0.0350 - val_mean_absolute_error: 0.1430 - val_mean_absolute_percentage_error: 21.6261 - val_root_mean_squared_error: 0.1872
Epoch 4/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0413 - mean_

In [ ]:
y_pred = model_hybrid_1.predict(x_test)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step


In [ ]:
loss, mae, mape, rmse = model_hybrid_1.evaluate(x_test, y_test)
print("model hybrid 1")
print("Mean Absolute Error:", mae)
print("Mean Square Error:", loss)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0299 - mean_absolute_error: 0.1320 - mean_absolute_percentage_error: 720959.9375 - root_mean_squared_error: 0.1724
model hybrid 1
Mean Absolute Error: 0.1431223750114441
Mean Square Error: 0.03406471014022827
Mean Absolute Percentage Error: 1441894.375
Root Mean Squared Error: 0.18456627428531647


## model 2 (CNN=64 LSTM=32)

In [ ]:
lr = 0.0003

model_hybrid_2 = Sequential()
model_hybrid_2.add(Conv1D(filters=64, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_hybrid_2.add(LSTM(32, activation='tanh', return_sequences=True))
model_hybrid_2.add(Dense(1))
model_hybrid_2.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_hybrid_2.summary()

Model: "sequential_22"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_18 (Conv1D)              │ (None, 7, 64)          │           768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_21 (LSTM)                  │ (None, 7, 32)          │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 7, 1)           │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,217 (51.63 KB)

 Trainable params: 13,217 (51.63 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_hybrid_2.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 52ms/step - loss: 0.2160 - mean_absolute_error: 0.4189 - mean_absolute_percentage_error: 74.2413 - root_mean_squared_error: 0.4623 - val_loss: 0.0659 - val_mean_absolute_error: 0.2062 - val_mean_absolute_percentage_error: 31.5760 - val_root_mean_squared_error: 0.2567
Epoch 2/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0742 - mean_absolute_error: 0.2229 - mean_absolute_percentage_error: 51.5422 - root_mean_squared_error: 0.2724 - val_loss: 0.0549 - val_mean_absolute_error: 0.1810 - val_mean_absolute_percentage_error: 27.3135 - val_root_mean_squared_error: 0.2342
Epoch 3/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0561 - mean_absolute_error: 0.1887 - mean_absolute_percentage_error: 40.7201 - root_mean_squared_error: 0.2368 - val_loss: 0.0466 - val_mean_absolute_error: 0.1659 - val_mean_absolute_percentage_error: 24.8434 - val_root_mean_squared_error: 0.2158
Epoch 4/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0448 - mean_

In [ ]:
y_pred = model_hybrid_2.predict(x_test)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step


In [ ]:
loss, mae, mape, rmse = model_hybrid_2.evaluate(x_test, y_test)
print("model hybrid 2")
print("Mean Absolute Error:", mae)
print("Mean Square Error:", loss)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0314 - mean_absolute_error: 0.1412 - mean_absolute_percentage_error: 829121.6250 - root_mean_squared_error: 0.1763
model hybrid 2
Mean Absolute Error: 0.15275104343891144
Mean Square Error: 0.03591059893369675
Mean Absolute Percentage Error: 1658214.375
Root Mean Squared Error: 0.18950092792510986


## model 3 (CNN=32 LSTM=32)

In [ ]:
lr = 0.0003

model_hybrid_3 = Sequential()
model_hybrid_3.add(Conv1D(filters=32, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_hybrid_3.add(LSTM(32, activation='tanh', return_sequences=True))
model_hybrid_3.add(Dense(1))
model_hybrid_3.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_hybrid_3.summary()

Model: "sequential_23"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_19 (Conv1D)              │ (None, 7, 32)          │           384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_22 (LSTM)                  │ (None, 7, 32)          │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 7, 1)           │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,737 (34.13 KB)

 Trainable params: 8,737 (34.13 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_hybrid_3.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 3s 43ms/step - loss: 0.1588 - mean_absolute_error: 0.3443 - mean_absolute_percentage_error: 62.9177 - root_mean_squared_error: 0.3963 - val_loss: 0.0671 - val_mean_absolute_error: 0.2209 - val_mean_absolute_percentage_error: 34.5943 - val_root_mean_squared_error: 0.2590
Epoch 2/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0753 - mean_absolute_error: 0.2216 - mean_absolute_percentage_error: 52.4081 - root_mean_squared_error: 0.2742 - val_loss: 0.0475 - val_mean_absolute_error: 0.1662 - val_mean_absolute_percentage_error: 24.8682 - val_root_mean_squared_error: 0.2180
Epoch 3/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0542 - mean_absolute_error: 0.1880 - mean_absolute_percentage_error: 40.6329 - root_mean_squared_error: 0.2328 - val_loss: 0.0353 - val_mean_absolute_error: 0.1436 - val_mean_absolute_percentage_error: 21.8351 - val_root_mean_squared_error: 0.1879
Epoch 4/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0429 - mean_

In [ ]:
y_pred = model_hybrid_3.predict(x_test)

3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 204ms/step


In [ ]:
loss, mae, mape, rmse = model_hybrid_3.evaluate(x_test, y_test)
print("model hybrid 3")
print("Mean Absolute Error:", mae)
print("Mean Square Error:", loss)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0306 - mean_absolute_error: 0.1396 - mean_absolute_percentage_error: 657109.4375 - root_mean_squared_error: 0.1742
model hybrid 3
Mean Absolute Error: 0.1504223495721817
Mean Square Error: 0.03487502783536911
Mean Absolute Percentage Error: 1314190.25
Root Mean Squared Error: 0.18674856424331665


## model 4 (CNN=24 LSTM=12)

In [ ]:
lr = 0.0003

model_hybrid_4 = Sequential()
model_hybrid_4.add(Conv1D(filters=24, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_hybrid_4.add(LSTM(12, activation='tanh', return_sequences=True))
model_hybrid_4.add(Dense(1))
model_hybrid_4.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_hybrid_4.summary()

Model: "sequential_24"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_20 (Conv1D)              │ (None, 7, 24)          │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_23 (LSTM)                  │ (None, 7, 12)          │         1,776 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 7, 1)           │            13 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,077 (8.11 KB)

 Trainable params: 2,077 (8.11 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_hybrid_4.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 3s 41ms/step - loss: 0.4096 - mean_absolute_error: 0.6138 - mean_absolute_percentage_error: 113.4865 - root_mean_squared_error: 0.6394 - val_loss: 0.2986 - val_mean_absolute_error: 0.5380 - val_mean_absolute_percentage_error: 81.3885 - val_root_mean_squared_error: 0.5465
Epoch 2/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.2243 - mean_absolute_error: 0.4391 - mean_absolute_percentage_error: 77.3612 - root_mean_squared_error: 0.4732 - val_loss: 0.1337 - val_mean_absolute_error: 0.3404 - val_mean_absolute_percentage_error: 50.7640 - val_root_mean_squared_error: 0.3656
Epoch 3/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.1077 - mean_absolute_error: 0.2812 - mean_absolute_percentage_error: 48.9930 - root_mean_squared_error: 0.3270 - val_loss: 0.0523 - val_mean_absolute_error: 0.1782 - val_mean_absolute_percentage_error: 26.9570 - val_root_mean_squared_error: 0.2287
Epoch 4/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0558 - mean

In [ ]:
y_pred = model_hybrid_4.predict(x_test)

3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 138ms/step


In [ ]:
loss, mae, mape, rmse = model_hybrid_4.evaluate(x_test, y_test)
print("model hybrid 4")
print("Mean Absolute Error:", mae)
print("Mean Square Error:", loss)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0318 - mean_absolute_error: 0.1435 - mean_absolute_percentage_error: 894792.2500 - root_mean_squared_error: 0.1772
model hybrid 4
Mean Absolute Error: 0.15604597330093384
Mean Square Error: 0.03708365187048912
Mean Absolute Percentage Error: 1789555.5
Root Mean Squared Error: 0.19257116317749023


## model 5 (CNN=24 LSTM=12)

In [ ]:
lr = 0.0003

model_hybrid_5 = Sequential()
model_hybrid_5.add(Conv1D(filters=256, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_hybrid_5.add(LSTM(128, activation='tanh', return_sequences=True))
model_hybrid_5.add(Dense(1))
model_hybrid_5.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_hybrid_5.summary()

Model: "sequential_25"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_21 (Conv1D)              │ (None, 7, 256)         │         3,072 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_24 (LSTM)                  │ (None, 7, 128)         │       197,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ (None, 7, 1)           │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 200,321 (782.50 KB)

 Trainable params: 200,321 (782.50 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_hybrid_5.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 3s 57ms/step - loss: 0.1554 - mean_absolute_error: 0.3318 - mean_absolute_percentage_error: 65.2598 - root_mean_squared_error: 0.3878 - val_loss: 0.0570 - val_mean_absolute_error: 0.1967 - val_mean_absolute_percentage_error: 28.7708 - val_root_mean_squared_error: 0.2388
Epoch 2/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 0.0482 - mean_absolute_error: 0.1760 - mean_absolute_percentage_error: 34.5001 - root_mean_squared_error: 0.2192 - val_loss: 0.0238 - val_mean_absolute_error: 0.1212 - val_mean_absolute_percentage_error: 19.1041 - val_root_mean_squared_error: 0.1544
Epoch 3/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 0.0284 - mean_absolute_error: 0.1315 - mean_absolute_percentage_error: 30.8105 - root_mean_squared_error: 0.1685 - val_loss: 0.0165 - val_mean_absolute_error: 0.1044 - val_mean_absolute_percentage_error: 16.0515 - val_root_mean_squared_error: 0.1286
Epoch 4/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.0247 - mean_

In [ ]:
y_pred = model_hybrid_4.predict(x_test)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step


In [ ]:
loss, mae, mape, rmse = model_hybrid_5.evaluate(x_test, y_test)
print("model hybrid 5")
print("Mean Absolute Error:", mae)
print("Mean Square Error:", loss)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0306 - mean_absolute_error: 0.1322 - mean_absolute_percentage_error: 600888.7500 - root_mean_squared_error: 0.1742
model hybrid 5
Mean Absolute Error: 0.14359210431575775
Mean Square Error: 0.03529002144932747
Mean Absolute Percentage Error: 1201751.375
Root Mean Squared Error: 0.18785639107227325


## model 6 (CNN=24 LSTM=12)

In [ ]:
lr = 0.0003

model_hybrid_6 = Sequential()
model_hybrid_6.add(Conv1D(filters=256, kernel_size=1, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model_hybrid_6.add(LSTM(64, activation='tanh', return_sequences=True))
model_hybrid_6.add(Dense(1))
model_hybrid_6.compile(loss='mse', optimizer='adam', metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.RootMeanSquaredError()])
model_hybrid_6.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_26"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_22 (Conv1D)              │ (None, 7, 256)         │         3,072 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_25 (LSTM)                  │ (None, 7, 64)          │        82,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 7, 1)           │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 85,313 (333.25 KB)

 Trainable params: 85,313 (333.25 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_hybrid_6.fit(
    x_train,
    y_train,
    validation_data=(x_val ,y_val),
    epochs=20,
    batch_size=42
)

Epoch 1/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 80ms/step - loss: 0.1779 - mean_absolute_error: 0.3537 - mean_absolute_percentage_error: 68.4362 - root_mean_squared_error: 0.4131 - val_loss: 0.0463 - val_mean_absolute_error: 0.1650 - val_mean_absolute_percentage_error: 24.6192 - val_root_mean_squared_error: 0.2151
Epoch 2/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - loss: 0.0480 - mean_absolute_error: 0.1741 - mean_absolute_percentage_error: 35.3113 - root_mean_squared_error: 0.2190 - val_loss: 0.0293 - val_mean_absolute_error: 0.1322 - val_mean_absolute_percentage_error: 20.5583 - val_root_mean_squared_error: 0.1711
Epoch 3/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 72ms/step - loss: 0.0349 - mean_absolute_error: 0.1473 - mean_absolute_percentage_error: 34.9322 - root_mean_squared_error: 0.1867 - val_loss: 0.0244 - val_mean_absolute_error: 0.1230 - val_mean_absolute_percentage_error: 18.2944 - val_root_mean_squared_error: 0.1562
Epoch 4/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0272 - mean_

In [ ]:
y_pred = model_hybrid_6.predict(x_test)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step


In [ ]:
loss, mae, mape, rmse = model_hybrid_6.evaluate(x_test, y_test)
print("model hybrid 5")
print("Mean Absolute Error:", mae)
print("Mean Square Error:", loss)
print("Mean Absolute Percentage Error:", mape)
print("Root Mean Squared Error:", rmse)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0327 - mean_absolute_error: 0.1349 - mean_absolute_percentage_error: 597823.8125 - root_mean_squared_error: 0.1798
model hybrid 5
Mean Absolute Error: 0.14960673451423645
Mean Square Error: 0.03958747908473015
Mean Absolute Percentage Error: 1195621.875
Root Mean Squared Error: 0.19896602630615234
